In [1]:
import sys
sys.path.append('/home/minjun/Jupyter/ocr/EAST/')
import numpy as np
import tensorflow as tf
import locality_aware_nms as nms_locality
import lanms
import model
from tensorflow.python.framework import graph_util
import tensorflow.contrib.slim as slim
import cv2

In [2]:
def load_pb(pb):
    with tf.gfile.GFile(pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)
    
def count_flops(graph,op_name):
    with tf.Session(graph=graph) as sess:
        total_parameters=0
        for variable in tf.trainable_variables():  
            local_parameters=1
            shape = variable.get_shape()  #getting shape of a variable
            for i in shape:
                local_parameters*=i.value  #mutiplying dimension values
            total_parameters+=local_parameters
        print("-----params-----" , total_parameters)
        sess.run(tf.global_variables_initializer())
        flops = tf.profiler.profile(sess.graph, options = tf.profiler.ProfileOptionBuilder.float_operation())
        flops_1 = flops.total_float_ops
        

        # ***** (2) freeze graph *****
        output_graph_def = graph_util.convert_variables_to_constants(sess, graph.as_graph_def(),op_name )
    with tf.gfile.GFile('tmp_graph.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())
    frozen_g = load_pb('./tmp_graph.pb')
    with frozen_g.as_default():
        flops = tf.profiler.profile(frozen_g, options = tf.profiler.ProfileOptionBuilder.float_operation())
        #print('FLOP after freezing', flops.total_float_ops)
        flops_2=flops.total_float_ops
    return flops_1,flops_2
def resize_image(im, max_side_len=1280):
    '''
    resize image to a size multiple of 32 which is required by the network
    :param im: the resized image
    :param max_side_len: limit of max image size to avoid out of memory in gpu
    :return: the resized image and the resize ratio
    '''
    h, w, _ = im.shape

    resize_w = w
    resize_h = h

    # limit the max side
    if max(resize_h, resize_w) > max_side_len:
        ratio = float(max_side_len) / resize_h if resize_h > resize_w else float(max_side_len) / resize_w
    else:
        ratio = max_side_len/max(resize_h, resize_w)
    resize_h = int(resize_h * ratio)
    resize_w = int(resize_w * ratio)

    resize_h = resize_h if resize_h % 32 == 0 else (resize_h // 32 - 1) * 32
    resize_w = resize_w if resize_w % 32 == 0 else (resize_w // 32 - 1) * 32
    resize_h = max(32, resize_h)
    resize_w = max(32, resize_w)
    im = cv2.resize(im, (int(resize_w), int(resize_h)))

    ratio_h = resize_h / float(h)
    ratio_w = resize_w / float(w)

    return im, (ratio_h, ratio_w)


In [3]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('backbone', 'Mobilenet', 'what kind of backbone')
tf.app.flags.DEFINE_string('f', '', 'kernel')
#FLAGS.backbone = 'Mobilenet' # Pvanet, Mobilenet, Resnet
#FLAGS.decoder = 'Expand' # original, CRAFT, Expand


In [ ]:
backbone = ['Mobilenet','Pvanet']
decoder = ['original','CRAFT','Expand']
comb = [(a,b) for a in backbone for b in decoder]


In [ ]:
for b,d in comb : 
    print('Encoder : {}, Decoder : {} '.format(b,d))
    FLAGS.backbone = b
    FLAGS.decoder = d
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=[1, 1280, 672, 3], name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())

    count_flops(g,[f_score.op.name,f_geometry.op.name])

In [ ]:
img_dir='/home/minjun/Jupyter/data/ocr/2015/test/img_1.jpg'
#img_dir='/home/minjun/Jupyter/data/ocr/2013/test/img_1.jpg'
#img_dir='/home/minjun/Jupyter/data/ocr/2017_MLT/test/ts_img_00001.jpg'
im = cv2.imread(img_dir)[:, :, ::-1]
im_resized, (ratio_h, ratio_w) = resize_image(im)
print(type(np.shape(im_resized)))
my_shape = [1]+list(np.shape(im_resized))
print(my_shape)

In [6]:
tf.reset_default_graph()
from nets import resnet_v1

    
#print(count_flops(g,[logits.op.name]))

In [7]:
FLAGS.backbone = 'Resnet'
FLAGS.decoder = 'original'
tf.reset_default_graph()
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=[1,224,224,3], name='input_images')
        with slim.arg_scope(resnet_v1.resnet_arg_scope(weight_decay=0.001)):
            logits, end_points = resnet_v1.resnet_v1_50(input_images, is_training=False, scope='resnet_v1_50')
        model_summary()


    print(count_flops(g,[logits.op.name]))


Instructions for updating:
keep_dims is deprecated, use keepdims instead
---------
Variables: name (type shape) [size]
---------
resnet_v1_50/conv1/weights:0 (float32_ref 7x7x3x64) [9408, bytes: 37632]
resnet_v1_50/conv1/BatchNorm/gamma:0 (float32_ref 64) [64, bytes: 256]
resnet_v1_50/conv1/BatchNorm/beta:0 (float32_ref 64) [64, bytes: 256]
resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/weights:0 (float32_ref 1x1x64x256) [16384, bytes: 65536]
resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/BatchNorm/gamma:0 (float32_ref 256) [256, bytes: 1024]
resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/BatchNorm/beta:0 (float32_ref 256) [256, bytes: 1024]
resnet_v1_50/block1/unit_1/bottleneck_v1/conv1/weights:0 (float32_ref 1x1x64x64) [4096, bytes: 16384]
resnet_v1_50/block1/unit_1/bottleneck_v1/conv1/BatchNorm/gamma:0 (float32_ref 64) [64, bytes: 256]
resnet_v1_50/block1/unit_1/bottleneck_v1/conv1/BatchNorm/beta:0 (float32_ref 64) [64, bytes: 256]
resnet_v1_50/block1/unit_1/bottleneck_v1/c

-----params----- 23508032
INFO:tensorflow:Froze 265 variables.
INFO:tensorflow:Converted 265 variables to const ops.
(7084412864, 6967138304)


In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.applications.mobilenet import MobileNet

run_meta = tf.RunMetadata()
with tf.Session(graph=tf.Graph()) as sess:
    K.set_session(sess)
    input_images = tf.placeholder(tf.float32, shape=[1,224,224,3], name='input_images')
    with slim.arg_scope(resnet_v1.resnet_arg_scope(weight_decay=0.001)):
        logits, end_points = resnet_v1.resnet_v1_50(input_images, is_training=False, scope='resnet_v1_50')

    opts = tf.profiler.ProfileOptionBuilder.float_operation()    
    flops = tf.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

    opts = tf.profiler.ProfileOptionBuilder.trainable_variables_parameter()    
    params = tf.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

    print("{:,} --- {:,}".format(flops.total_float_ops, params.total_parameters))

In [ ]:
from research.slim.nets.mobilenet import mobilenet_v2
tf.reset_default_graph()
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=[1,224,224,3], name='input_images')
        with slim.arg_scope(mobilenet_v2.training_scope(weight_decay=weight_decay)):
            logits, endpoints = mobilenet_v2.mob(images,is_training=is_training)
        model_summary()


    print(count_flops(g,[logits.op.name]))

In [ ]:
for k in end_points.keys():
    print(k)

In [ ]:
for v in end_points.values():
    print(v)

In [ ]:
print(end_points['resnet_v1_50/block1/unit_2/bottleneck_v1/conv3'])
print(end_points['resnet_v1_50/block2/unit_3/bottleneck_v1/conv3'])
print(end_points['resnet_v1_50/block3/unit_5/bottleneck_v1/conv3'])
print(end_points['resnet_v1_50/block4/unit_3/bottleneck_v1/conv3'])

In [ ]:
logits

In [ ]:
FLAGS.backbone = 'Resnet'
FLAGS.decoder = 'original2'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=[1,1280,704,3], name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)
        print(len(graph.get_operations()))
        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()

    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'original'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        
    #print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand8'
tf.reset_default_graph()
input_images = tf.placeholder(tf.float32, shape=[1,256,256,3], name='input_images')
global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
f_score, f_geometry = model.model(input_images, is_training=False)




In [ ]:
for l in tf.trainable_variables():
    print(l.initial_value)


In [ ]:
l.initial_value

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'CRAFT'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=my_shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)
        print(len(graph.get_operations()))
        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Pvanet'
FLAGS.decoder = 'original'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=my_shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)
        print(len(graph.get_operations()))
        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=my_shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand8'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=my_shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)
        print(len(graph.get_operations()))
        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand7'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=[1, 512, 512, 3], name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand5'
shape = [1,1280, 672,3]
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand4'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand3'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'Expand2'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Resnet'
FLAGS.decoder = 'original'
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=[1, 512, 512, 3], name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        #model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
FLAGS.backbone = 'Mobilenet'
FLAGS.decoder = 'CRAFT'
shape = [1,1280, 672,3]
with tf.device('/gpu:5' ):
    g = tf.Graph()
    with g.as_default() as graph:
        input_images = tf.placeholder(tf.float32, shape=shape, name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())
        model_summary()


    print(count_flops(g,[f_score.op.name,f_geometry.op.name]))

In [ ]:
from research.slim.nets.mobilenet import mobilenet_v2

with slim.arg_scope(mobilenet_v2.training_scope(weight_decay=1e-5)):
    input_images = tf.placeholder(tf.float32, shape=[1, 512, 512, 3], name='input_images')
    logits, endpoints = mobilenet_v2.mobilenet_base(input_images,is_training=False)

In [ ]:
end_points = dict()
end_points['pool2'] = endpoints['layer_4/output']
end_points['pool3'] = endpoints['layer_6/output']
end_points['pool4'] = endpoints['layer_13/output']
end_points['pool5'] = endpoints['layer_19']

In [ ]:
end_points